In [1]:
%load_ext autoreload
# %reload_ext autoreload

# Reload all modules imported with %aimport every time before executing the Python code typed
%autoreload 1

%aimport context_nn 
%aimport phrase_feeder 
%aimport bitnotes
%aimport watch_point
%aimport cluster
%aimport constants
import numpy as np
from bitarray import bitarray
from context_nn import ContextNN
from watch_point import WatchPoint
from cluster import Cluster
from phrase_feeder import PhraseFeeder
from bitnotes import BitNotes
from pprint import pprint
import math
import constants as const

In [2]:
import pickle

def load_cxnn(file_name: str) -> ContextNN:
    with open(file_name, 'rb') as f:
        cxnn = pickle.load(f)
        return cxnn
    
def load_noted_phrases(file_name: str) -> list:
    with open(file_name, 'rb') as f:
        noted_phrases = pickle.load(f)
        return noted_phrases

In [91]:
cxnn = load_cxnn('./data/cxnn_langs_2000.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_eng.pickle')

In [36]:
cxnn = load_cxnn('./data/cxnn_monotone.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_bbb.pickle')

In [ ]:
cxnn = load_cxnn('./data/cxnn_probs.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_3331.pickle')

In [44]:
cxnn = load_cxnn('./data/cxnn_dna.pickle')
noted_phrases = load_noted_phrases('./data/texts/tst_cdef.pickle')

In [92]:
%%time

cxnn.start_detection()

for num, phrase in enumerate(noted_phrases):
    bit_chord = cxnn.bit_notes.phrase_chord_as_bits(phrase)
    cxnn.detect_bits(bit_chord)
    if num % 500 == 0:
        print(f'phrase # {num}/{len(noted_phrases)}')

phrase # 0/2330
phrase # 500/2330
phrase # 1000/2330
phrase # 1500/2330
phrase # 2000/2330
Wall time: 55.7 s


In [62]:
print(cxnn.cluster_count())
pprint(cxnn.cluster_len_stats())
pprint(cxnn.cluster_activity_stats())

15719
{6: 1067, 7: 6009, 8: 6892, 9: 1425, 10: 277, 11: 43, 12: 6}
{0: 15719}


In [15]:
cxnn.data_marks

{(0,): 'bel',
 (1,): 'blg',
 (2,): 'eng',
 (3,): 'epo',
 (4,): 'jbo',
 (5,): 'pol',
 (6,): 'rus',
 (7,): 'ukr'}

In [93]:
bit_detections = {}
bit_clusters = {}
for point in cxnn.point_objects:
    key = point.output_bit
    bit_detections[key] = bit_detections.get(key, 0) + point.detections
    bit_clusters[key] = bit_clusters.get(key, 0) + point.cluster_count()
    
print(sorted(bit_detections.items()))
print(sorted(bit_detections.items(), key=lambda x: x[1], reverse=True))

result = [(bit, bit_detections[bit] / bit_clusters[bit]) for bit in bit_clusters.keys()]

print(sorted(result, key=lambda x: x[1], reverse=True))




[(0, 198537), (1, 161732), (2, 967938), (3, 151854), (4, 500263), (5, 420421), (6, 79968), (7, 90712)]
[(2, 967938), (4, 500263), (5, 420421), (0, 198537), (1, 161732), (3, 151854), (7, 90712), (6, 79968)]
[(2, 91.30629185925856), (1, 76.90537327627199), (6, 69.41666666666667), (0, 61.314700432365655), (7, 49.27322107550244), (3, 48.391969407265776), (5, 36.302650893705206), (4, 32.5543697533676)]


In [ ]:
detection = cxnn.summarize_detection()

In [ ]:
for bits, label in cxnn.data_marks.items():
    rate = np.sum(detection[list(bits)])
    print(label, rate)

In [21]:
cxnn.vectors_received

2500

In [69]:
print(cxnn.cluster_count())
points = [(bit, count) for bit, count in cxnn.point_stats() if count > 0]
# print(sorted(points, key=lambda x: x[0]))
bit_points = {}
for bit, count in points:
    bit_points[bit] = bit_points.get(bit, 0) + 1

print(sorted(bit_points.items()))
    
bit_clusters = {}
for bit, count in points:
    bit_clusters[bit] = bit_clusters.get(bit, 0) + count

print(sorted(bit_clusters.items()))

bit_activations = {}
for point in cxnn.point_objects:
    key = point.output_bit
    bit_activations[key] = bit_activations.get(key, 0) + sum(cluster.activation_count() 
                                                             for cluster in point.cluster_objects 
                                                             if cluster.consolidations >= 0 and 
                                                                cluster.activation_count() > 200)

print(sorted(bit_activations.items()))

bit_divs = {}
for key in bit_clusters.keys():
    bit_divs[key] = bit_activations[key] / (bit_clusters[key] * bit_points[key])

pprint(sorted(bit_divs.items()))


15719
[(0, 187), (1, 151), (2, 144), (3, 154), (4, 202), (5, 179), (6, 104), (7, 108)]
[(0, 2390), (1, 1061), (2, 2650), (3, 914), (4, 2513), (5, 5279), (6, 407), (7, 505)]
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0)]
[(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0)]


In [8]:
26307 / 256

102.76171875

In [9]:
231968 / 2645

87.70056710775047

In [5]:
1542230 / 2649

582.1932804832012

In [6]:
521326 / 2649

196.8010570026425